##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 訓練您的第一個神經網絡: 基本分類

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/zh-tw/r1/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/zh-tw/r1/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

本指南訓練了一個神經網絡模型，來對服裝圖像進行分類，例如運動鞋和襯衫。如果您不了解所有細節也不需要擔心，這是一個對完整TensorFlow項目的簡要概述，相關的細節會在需要時進行解釋

本指南使用[tf.keras](https://www.tensorflow.org/r1/guide/keras)，這是一個用於在TensorFlow中構建和訓練模型的高級API。

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# 導入TensorFlow和tf.keras
import tensorflow as tf
from tensorflow import keras

# 導入輔助庫
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## 導入Fashion MNIST數據集

本指南使用[Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) 數據集，其中包含了10個類別中共70,000張灰度圖像。圖像包含了低分辨率（28 x 28像素）的單個服裝物品，如下所示:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST 旨在替代傳統的[MNIST](http://yann.lecun.com/exdb/mnist/)數據集 — 它經常被作為機器學習在計算機視覺方向的"Hello, World"。MNIST數據集包含手寫數字（0,1,2等）的圖像，其格式與我們在此處使用的服裝相同。

本指南使用Fashion MNIST進行多樣化，因為它比普通的MNIST更具挑戰性。兩個數據集都相對較小，用於驗證算法是否按預期工作。它們是測試和調試代碼的良好起點。

我們將使用60,000張圖像來訓練網絡和10,000張圖像來評估網絡模型學習圖像分類任務的準確程度。您可以直接從TensorFlow使用Fashion MNIST，只需導入並加載數據

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

加載數據集並返回四個NumPy數組:

* `train_images`和`train_labels`數組是*訓練集*—這是模型用來學習的數據。
* 模型通過*測試集*進行測試, 即`test_images`與 `test_labels`兩個數組。

圖像是28x28 NumPy數組，像素值介於0到255之間。*labels*是一個整數數組，數值介於0到9之間。這對應了圖像所代表的服裝的*類別*:

<table>
  <tr>
    <th>標籤</th>
    <th>類別</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

每個圖像都映射到一個標籤。由於*類別名稱*不包含在數據集中,因此把他們存儲在這里以便在繪製圖像時使用:

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## 探索數據

讓我們在訓練模型之前探索數據集的格式。以下顯示訓練集中有60,000個圖像，每個圖像表示為28 x 28像素:

In [0]:
train_images.shape

同樣，訓練集中有60,000個標籤:

In [0]:
len(train_labels)

每個標籤都是0到9之間的整數:

In [0]:
train_labels

測試集中有10,000個圖像。同樣，每個圖像表示為28×28像素:

In [0]:
test_images.shape

測試集包含10,000個圖像標籤:

In [0]:
len(test_labels)

## 數據預處理

在訓練網絡之前必須對數據進行預處理。如果您檢查訓練集中的第一個圖像，您將看到像素值落在0到255的範圍內:

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

在饋送到神經網絡模型之前，我們將這些值縮放到0到1的範圍。為此，我們將像素值值除以255。重要的是，對*訓練集*和*測試集*要以相同的方式進行預處理:

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

顯示*訓練集*中的前25個圖像，並在每個圖像下方顯示類名。驗證數據格式是否正確，我們是否已準備好構建和訓練網絡。

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## 構建模型

構建神經網絡需要配置模型的層，然後編譯模型。

### 設置網絡層

一個神經網絡最基本的組成部分便是*網絡層*。網絡層從提供給他們的數據中提取表示，並期望這些表示對當前的問題更加有意義

大多數深度學習是由串連在一起的網絡層所組成。大多數網絡層，例如`tf.keras.layers.Dense`，具有在訓練期間學習的參數。

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

網絡中的第一層, `tf.keras.layers.Flatten`, 將圖像格式從一個二維數組(包含著28x28個像素)轉換成為一個包含著28 * 28 = 784個像素的一維數組。可以將這個網絡層視為它將圖像中未堆疊的像素排列在一起。這個網絡層沒有需要學習的參數;它僅僅對數據進行格式化。

在像素被展平之後，網絡由一個包含有兩個`tf.keras.layers.Dense`網絡層的序列組成。他們被稱作稠密鏈接層或全連接層。第一個`Dense`網絡層包含有128個節點(或被稱為神經元)。第二個(也是最後一個)網絡層是一個包含10個節點的*softmax*層—它將返回包含10個概率分數的數組，總和為1。每個節點包含一個分數，表示當前圖像屬於10個類別之一的概率。

### 編譯模型

在模型準備好進行訓練之前，它還需要一些配置。這些是在模型的*編譯(compile)*步驟中添加的:

* *損失函數* —這可以衡量模型在培訓過程中的準確程度。我們希望將此函數最小化以"驅使"模型朝正確的方向擬合。
* *優化器* —這就是模型根據它看到的數據及其損失函數進行更新的方式。
* *評價方式* —用於監控訓練和測試步驟。以下示例使用*準確率(accuracy)*，即正確分類的圖像的分數。

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 訓練模型

訓練神經網絡模型需要以下步驟:

1. 將訓練數據提供給模型 - 在本案例中，他們是`train_images`和`train_labels`數組。
2. 模型學習如何將圖像與其標籤關聯
3. 我們使用模型對測試集進行預測, 在本案例中為`test_images`數組。我們驗證預測結果是否匹配`test_labels`數組中保存的標籤。

通過調用`model.fit`方法來訓練模型 — 模型對訓練數據進行"擬合"。

In [0]:
model.fit(train_images, train_labels, epochs=5)

隨著模型訓練，將顯示損失和準確率等指標。該模型在訓練數據上達到約0.88(或88％)的準確度。

## 評估準確率

接下來，比較模型在測試數據集上的執行情況:

In [0]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('Test accuracy:', test_acc)

事實證明，測試數據集的準確性略低於訓練數據集的準確性。訓練精度和測試精度之間的差距是*過擬合*的一個例子。過擬合是指機器學習模型在新數據上的表現比在訓練數據上表現更差。

## 進行預測

通過訓練模型，我們可以使用它來預測某些圖像。

In [0]:
predictions = model.predict(test_images)

在此，模型已經預測了測試集中每個圖像的標籤。我們來看看第一個預測:

In [0]:
predictions[0]

預測是10個數字的數組。這些描述了模型的"信心"，即圖像對應於10種不同服裝中的每一種。我們可以看到哪個標籤具有最高的置信度值：

In [0]:
np.argmax(predictions[0])

因此，模型最有信心的是這個圖像是ankle boot，或者 `class_names[9]`。我們可以檢查測試標籤，看看這是否正確:

In [0]:
test_labels[0]

我們可以用圖表來查看全部10個類別

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img, cmap=plt.cm.binary)
  
  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

讓我們看看第0個圖像，預測和預測數組。

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

讓我們繪製幾個圖像及其預測結果。正確的預測標籤是藍色的，不正確的預測標籤是紅色的。該數字給出了預測標籤的百分比(滿分100)。請注意，即使非常自信，也可能出錯。

In [0]:
# 繪製前X個測試圖像，預測標籤和真實標籤
# 以藍色顯示正確的預測，紅色顯示不正確的預測
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()

最後，使用訓練的模型對單個圖像進行預測。

In [0]:
# 從測試數據集中獲取圖像
img = test_images[0]

print(img.shape)

`tf.keras`模型經過優化，可以一次性對*批量*,或者一個集合的數據進行預測。因此，即使我們使用單個圖像，我們也需要將其添加到列表中:

In [0]:
# 將圖像添加到批次中，即使它是唯一的成員。
img = (np.expand_dims(img,0))

print(img.shape)

現在來預測圖像:

In [0]:
predictions_single = model.predict(img)

print(predictions_single)

In [0]:
plot_value_array(0, predictions_single, test_labels)
plt.xticks(range(10), class_names, rotation=45)
plt.show()

`model.predict`返回一個包含列表的列表，每個圖像對應一個列表的數據。獲取批次中我們(僅有的)圖像的預測:

In [0]:
prediction_result = np.argmax(predictions_single[0])
print(prediction_result)

而且，和之前一樣，模型預測標籤為9。